In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
%matplotlib inline


In [2]:
# Reading whale returns
whale_returns_csv = Path("Resources/whale_returns.csv")
whale_returns = pd.read_csv(whale_returns_csv,index_col="Date", parse_dates=True)
whale_returns.head()


FileNotFoundError: [Errno 2] File Resources\whale_returns.csv does not exist: 'Resources\\whale_returns.csv'

In [ ]:
# Count nulls
whale_returns.isnull().sum()


In [ ]:
# Drop nulls
whale_returns.dropna(inplace=True)
whale_returns.isnull().sum()


In [ ]:
# Reading algorithmic returns
algo_returns_csv = Path("./Resources/algo_returns.csv")
algo_returns = pd.read_csv(algo_returns_csv,index_col="Date", parse_dates=True)
algo_returns.head()


In [ ]:
# Count nulls
algo_returns.isnull().sum()


In [ ]:
# Drop nulls
algo_returns.dropna(inplace=True)
algo_returns.isnull().sum()


In [ ]:
# Reading S&P 500 Closing Prices
sp500_history_csv = Path("Resources/sp500_history.csv")
sp500_closingprices = pd.read_csv(sp500_history_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)
sp500_closingprices.head()


In [ ]:
# Check Data Types
sp500_closingprices.dtypes


In [ ]:
# Fix Data Types
sp500_closingprices["Close"] = sp500_closingprices["Close"].str.replace("$","")
sp500_closingprices["Close"] = sp500_closingprices["Close"].astype("float")
sp500_closingprices.dtypes


In [ ]:

# Calculate Daily Returns
sp500_dailyreturns = sp500_closingprices.pct_change()
sp500_dailyreturns.head()


In [ ]:
# Drop nulls
sp500_dailyreturns.dropna(inplace=True)
sp500_dailyreturns.head()


In [ ]:
# Rename Column
sp500_dailyreturns = sp500_dailyreturns.rename(columns={"Close":   "S&P 500 Close"})
sp500_dailyreturns


In [ ]:
# Concatenate all DataFrames into a single DataFrame

combined_df = pd.concat([whale_returns, algo_returns, sp500_dailyreturns], axis="columns", join="inner")
combined_df.head()


In [ ]:
# Plot daily returns
combined_df.plot(figsize=(20,10), title="Daily Returns")


In [ ]:
# Plot cumulative returns
cumulative_returns = (1 + combined_df).cumprod() - 1
cumulative_returns.plot(figsize=(20,10), title="Cumulative Returns")

"All portfolios outperformed the S and P"

In [ ]:
# Box plot to visually show risk
combined_df.plot(kind="box", figsize=(20,10), title="Portfolio Risk")


In [ ]:
# Daily Standard Deviations
# Calculate the standard deviation for each portfolio. 
portfolio_risk = combined_df.std()
portfolio_risk
# Which portfolios are riskier than the S&P 500? Tiger and Berkshire


In [ ]:
# Determine which portfolios are riskier than the S&P 500
SP_500_Risk = portfolio_risk[-1]
portfolio_risk.to_frame()
portfolio_risk.apply(lambda x : True if x> SP_500_Risk else False)


In [ ]:

# Calculate the annualized standard deviation (252 trading days)
annualized_sd = combined_df.std() * np.sqrt (252)
annualized_sd


In [ ]:
# Calculate and plot the rolling standard deviation for
# the S&P 500 and whale portfolios using a 21 trading day window
combined_df["S&P 500 Close"].rolling(window=21).std().plot(figsize=(15,8))


In [ ]:
# Construct a correlation table
price_correlation = combined_df.corr()
price_correlation


In [ ]:
# Calculate Beta for a single portfolio compared to the total market (S&P 500)
covariance = combined_df['SOROS FUND MANAGEMENT LLC'].cov(combined_df['S&P 500 Close'])
variance = combined_df['S&P 500 Close'].var()
sorosbeta = covariance/variance
sorosbeta


In [ ]:

# Calculate a rolling window using the exponentially weighted moving average. 
combined_df.ewm(halflife=21)


In [ ]:
# Calculate annualized Sharpe Ratios
sharpe_ratios = (combined_df.mean() * 252) / (combined_df.std() * np.sqrt(252))
sharpe_ratios


In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpe_ratios.plot.bar()


In [ ]:
# Read the first stock
goog_historical_csv = Path("Resources/goog_historical.csv")
goog_historical = pd.read_csv(goog_historical_csv, index_col="Trade DATE", infer_datetime_format=True, parse_dates=True)
goog_historical.head()


In [ ]:
# Read the second stock
aapl_historical_csv = Path("Resources/aapl_historical.csv")
aapl_historical = pd.read_csv(aapl_historical_csv, index_col="Trade DATE", infer_datetime_format=True, parse_dates=True)
aapl_historical.head()


In [ ]:
# Read the third stock
cost_historical_csv = Path("Resources/cost_historical.csv")
cost_historical = pd.read_csv(cost_historical_csv, index_col="Trade DATE", infer_datetime_format=True, parse_dates=True)
cost_historical.head()


In [ ]:
# Concatenate all stocks into a single DataFrame
stock_df = pd.concat([aapl_historical, cost_historical, goog_historical], axis="rows", join="outer")
stock_df.head()


In [ ]:
# Reset the index
stock_df.reset_index()


In [ ]:
# Pivot so that each column of prices represents a unique symbol
stock_df = stock_df.pivot(columns="Symbol", values="NOCP")
stock_df.head()


In [ ]:

# Drop Nulls
stock_daily_returns = stock_df.pct_change()
stock_daily_returns.dropna(inplace=True)
stock_daily_returns.head()


In [ ]:
# Calculate weighted portfolio returns
weights = [1/3, 1/3, 1/3]
weighted_returns = stock_daily_returns.dot(weights)
weighted_returns.head()


In [ ]:
# Add your "Custom" portfolio to the larger dataframe of fund returns
my_portfolio_returns = pd.concat([combined_df, weighted_returns], axis="columns", join="inner")
x = my_portfolio_returns.columns[-1]


In [ ]:
my_portfolio_returns = my_portfolio_returns.rename(columns={x:"Custom"})
my_portfolio_returns.columns


In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)
my_portfolio_returns.isnull()
my_portfolio_returns.dropna(axis=0, how="any", inplace=True)
my_portfolio_returns


In [ ]:
# Risk
my_portfolio_risk = my_portfolio_returns.std() * np.sqrt (252)
my_portfolio_risk


In [ ]:
# Rolling
my_portfolio_returns.rolling(window=21).std().plot(figsize=(20,10), title="21 Day Rolling Standard Deviation")


In [ ]:
# Annualized Sharpe Ratios
sharpe_ratios_with_custom = (my_portfolio_returns.mean() * 252) / (my_portfolio_returns.std() * np.sqrt(252))
sharpe_ratios_with_custom


In [ ]:
# Beta
covariance_all = my_portfolio_returns['Custom'].cov(my_portfolio_returns['S&P 500 Close'])
covariance_all
variance_all = my_portfolio_returns['S&P 500 Close'].var()
berkshire_beta = covariance / variance
rolling_covariance_all = my_portfolio_returns['Custom'].rolling(window=60).cov(my_portfolio_returns['S&P 500 Close'])
rolling_variance_all = my_portfolio_returns['S&P 500 Close'].rolling(window=60).var()
rolling_beta_all = rolling_covariance_all / rolling_variance_all
rolling_beta_all.plot(figsize=(20, 10), title='Custom Portfolio Beta')
